In [1]:
import torch
import pickle
import dgl
import torch.nn.functional as F
import numpy as np

d:\Software\anaconda3\envs\gnn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task = 0
seed = 1
dataset = 'cora'

In [3]:
feat_syn = torch.load('cgnn/feat_reddit_0.01_{}_{}.pt'.format(seed,task))
label_syn = torch.load('cgnn/labels_reddit_0.01_{}_{}.pt'.format(seed,task))
adj_syn = torch.load('cgnn/adj_reddit_0.01_{}_{}.pt'.format(seed,task))
# with open('cgnn/data_reddit_0.01_{}_{}.pkl'.format(seed,task),'rb') as f:
#     data = pickle.load(f)
# with open('cgnn/taskcla_reddit_0.01_{}_{}.pkl'.format(seed,task),'rb') as f:
#     taskcla = pickle.load(f)

In [4]:
# idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
# adj, features, labels = data.adj, data.features, data.labels

In [5]:
label_syn.unique()

tensor([0, 1, 2, 3, 4, 5, 6, 7], device='cuda:0')

In [6]:
device = 'cuda:0'
adj_syn[adj_syn < 0.05] = 0
g_selfloop = adj_syn + torch.eye(adj_syn.shape[0]).to(device)
edge_index = g_selfloop.nonzero().T
g_syn = dgl.graph((edge_index[0],edge_index[1])).to(device)
edge_weight = g_selfloop[edge_index[0],edge_index[1]]
feat_syn = feat_syn.to(device)
label_syn = label_syn.to(device)

In [7]:
ce = torch.nn.CrossEntropyLoss()

In [8]:
from models.GAT import NET
model = NET(feat_syn.shape[1], 32, heads = 8, num_layers = 2, n_class = 8).to(device)
# from models.GCN import NET
# model = NET(feat_syn.shape[1], 256, n_layers = 2, n_class = taskcla[1]).to(device)
optimizer_model = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

In [9]:
model.train()
for _ in range(6000):
    optimizer_model.zero_grad()
    # logits = model(g_syn, feat_syn)
    logits = model(g_syn, feat_syn, edge_weight = edge_weight)
    # logits = F.log_softmax(logits,dim=1)
    # loss = F.nll_loss(logits, label_syn)
    loss = ce(logits, label_syn)
    loss.backward()
    optimizer_model.step()
    print(loss.item(),end='\r')

: 

In [69]:
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
adj, features, labels = data.adj, data.features, data.labels
# adj_test = adj[np.ix_(idx_test, idx_test)]
adj_test = adj
g = dgl.from_scipy(adj_test).to(device)
g = dgl.add_self_loop(g)
features = torch.Tensor(features).to(device)
labels = torch.LongTensor(labels).to(device)
idx_train = torch.LongTensor(idx_train).to(device)
idx_val = torch.LongTensor(idx_val).to(device)
idx_test = torch.LongTensor(idx_test).to(device)

AttributeError: 'list' object has no attribute 'idx_train'

In [11]:
with torch.no_grad():
    logits = model(g, features)
    prob, prediction = torch.max(logits[idx_test], dim=1)
    prediction = prediction.cpu().numpy()
    truth = labels[idx_test].cpu().numpy()

In [12]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(truth, prediction)
print(acc)

0.907668086682719


In [70]:
import random
import numpy as np
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

from data.load_data import *
data, taskcla, in_feat = load_dataset('reddit', device)

In [71]:
g, features, _, labels, train_mask, val_mask, test_mask = data[task].retrieve_data()

In [72]:
with torch.no_grad():
    logits = model(g, features)
    prob, prediction = torch.max(logits[test_mask], dim=1)
    # prediction = prediction[idx_test].cpu().numpy()
    # truth = labels[idx_test].cpu().numpy()
    prediction = prediction.cpu().numpy()
    truth = labels[test_mask].cpu().numpy()

In [73]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(truth, prediction)
print(acc)

0.8511017704750987


: 